In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras
from keras.layers import Input, Lambda, Dense, Flatten
from keras.applications import ResNet101
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
IMAGE_SIZE = [224, 224]

train_path = '/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumor Training'
test_path = '/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumour Testing'

In [4]:
base_model = ResNet101(weights='imagenet', include_top=False, input_shape=IMAGE_SIZE + [3])

In [5]:
for layer in base_model.layers:
    layer.trainable = False

In [6]:
folders = glob('/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumor Training/*')
x = base_model.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(len(folders), activation='relu')(x)
# and a logistic layer
predictions = Dense(len(folders), activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

                                                                                                  
 conv2_block3_1_conv (Conv2  (None, 56, 56, 64)           16448     ['conv2_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv2_block3_1_bn (BatchNo  (None, 56, 56, 64)           256       ['conv2_block3_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv2_block3_1_relu (Activ  (None, 56, 56, 64)           0         ['conv2_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv2_blo

 ation)                                                                                           
                                                                                                  
 conv3_block2_3_conv (Conv2  (None, 28, 28, 512)          66048     ['conv3_block2_2_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block2_3_bn (BatchNo  (None, 28, 28, 512)          2048      ['conv3_block2_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block2_add (Add)      (None, 28, 28, 512)          0         ['conv3_block1_out[0][0]',    
                                                                     'conv3_block2_3_bn[0][0]']   
          

 conv4_block1_2_conv (Conv2  (None, 14, 14, 256)          590080    ['conv4_block1_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv4_block1_2_bn (BatchNo  (None, 14, 14, 256)          1024      ['conv4_block1_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block1_2_relu (Activ  (None, 14, 14, 256)          0         ['conv4_block1_2_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv4_block1_0_conv (Conv2  (None, 14, 14, 1024)         525312    ['conv3_block4_out[0][0]']    
 D)       

                                                                                                  
 conv4_block3_out (Activati  (None, 14, 14, 1024)         0         ['conv4_block3_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv4_block4_1_conv (Conv2  (None, 14, 14, 256)          262400    ['conv4_block3_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv4_block4_1_bn (BatchNo  (None, 14, 14, 256)          1024      ['conv4_block4_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_blo

...

 ion)                                                                                             
                                                                                                  
 conv4_block15_1_conv (Conv  (None, 14, 14, 256)          262400    ['conv4_block14_out[0][0]']   
 2D)                                                                                              
                                                                                                  
 conv4_block15_1_bn (BatchN  (None, 14, 14, 256)          1024      ['conv4_block15_1_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_block15_1_relu (Acti  (None, 14, 14, 256)          0         ['conv4_block15_1_bn[0][0]']  
 vation)                                                                                          
          

 conv4_block17_3_bn (BatchN  (None, 14, 14, 1024)         4096      ['conv4_block17_3_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_block17_add (Add)     (None, 14, 14, 1024)         0         ['conv4_block16_out[0][0]',   
                                                                     'conv4_block17_3_bn[0][0]']  
                                                                                                  
 conv4_block17_out (Activat  (None, 14, 14, 1024)         0         ['conv4_block17_add[0][0]']   
 ion)                                                                                             
                                                                                                  
 conv4_block18_1_conv (Conv  (None, 14, 14, 256)          262400    ['conv4_block17_out[0][0]']   
 2D)      

                                                                                                  
 conv4_block20_2_relu (Acti  (None, 14, 14, 256)          0         ['conv4_block20_2_bn[0][0]']  
 vation)                                                                                          
                                                                                                  
 conv4_block20_3_conv (Conv  (None, 14, 14, 1024)         263168    ['conv4_block20_2_relu[0][0]']
 2D)                                                                                              
                                                                                                  
 conv4_block20_3_bn (BatchN  (None, 14, 14, 1024)         4096      ['conv4_block20_3_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_blo

 vation)                                                                                          
                                                                                                  
 conv4_block23_2_conv (Conv  (None, 14, 14, 256)          590080    ['conv4_block23_1_relu[0][0]']
 2D)                                                                                              
                                                                                                  
 conv4_block23_2_bn (BatchN  (None, 14, 14, 256)          1024      ['conv4_block23_2_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_block23_2_relu (Acti  (None, 14, 14, 256)          0         ['conv4_block23_2_bn[0][0]']  
 vation)                                                                                          
          

 conv5_block2_add (Add)      (None, 7, 7, 2048)           0         ['conv5_block1_out[0][0]',    
                                                                     'conv5_block2_3_bn[0][0]']   
                                                                                                  
 conv5_block2_out (Activati  (None, 7, 7, 2048)           0         ['conv5_block2_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv5_block3_1_conv (Conv2  (None, 7, 7, 512)            1049088   ['conv5_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv5_block3_1_bn (BatchNo  (None, 7, 7, 512)            2048      ['conv5_block3_1_conv[0][0]'] 
 rmalizati

In [7]:
# compile the model
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [9]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)




# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumor Training',
                                                 target_size = (224, 224),
                                                 batch_size = 40,
                                                 class_mode = 'categorical')




test_set = test_datagen.flow_from_directory('/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumour Testing',
                                            target_size = (224, 224),
                                            batch_size = 40,
                                            class_mode = 'categorical')

Found 4600 images belonging to 2 classes.
Found 1872 images belonging to 2 classes.


In [10]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/10
115/115 [==============================] - 440s 4s/step - loss: 0.6996 - accuracy: 0.5120 - val_loss: 0.6920 - val_accuracy: 0.5449
Epoch 2/10
115/115 [==============================] - 483s 4s/step - loss: 0.6911 - accuracy: 0.5463 - val_loss: 0.6906 - val_accuracy: 0.5449
Epoch 3/10
115/115 [==============================] - 482s 4s/step - loss: 0.6901 - accuracy: 0.5463 - val_loss: 0.6899 - val_accuracy: 0.5449
Epoch 4/10
115/115 [==============================] - 513s 4s/step - loss: 0.6895 - accuracy: 0.5463 - val_loss: 0.6895 - val_accuracy: 0.5449
Epoch 5/10
115/115 [==============================] - 524s 5s/step - loss: 0.6892 - accuracy: 0.5463 - val_loss: 0.6893 - val_accuracy: 0.5449
Epoch 6/10
115/115 [==============================] - 572s 5s/step - loss: 0.6890 - accuracy: 0.5463 - val_loss: 0.6892 - val_accuracy: 0.5449
Epoch 7/10
115/115 [==============================] - 616s 5s/step - loss: 0.6889 - accuracy: 0.5463 - val_loss: 0.6891 - val_accuracy: 0.5449